In [2]:
pip install git+https://github.com/alberanid/imdbpy

  Cloning https://github.com/alberanid/imdbpy to /tmp/pip-req-build-0gsavfv6
  Running command git clone -q https://github.com/alberanid/imdbpy /tmp/pip-req-build-0gsavfv6
  Created wheel for IMDbPY: filename=IMDbPY-2021.10.24-py3-none-any.whl size=300551 sha256=dc1d70c6ba2992eade60d1ed35ad48ec2bafb2c3554ff72fae25f8c554d8b892
  Stored in directory: /tmp/pip-ephem-wheel-cache-f41n9qen/wheels/d1/75/06/4d048b85021fb5f979ece79f70f27550ed04b76f9603dcb0d9
Successfully built IMDbPY


In [3]:
import json
import spacy
import re
import nltk
import difflib
from nltk.tokenize import word_tokenize
from nltk import pos_tag, ne_chunk
from nltk.corpus import stopwords
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('stopwords')
from imdb import IMDb
import math
from collections import Counter
WORD = re.compile(r'\w+')
ia = IMDb()

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [209]:
mystop_words = stopwords.words('english')
mystop_words.extend(['performance','made','-','congrats','congratulations','got','knows','well','well-','hell','yeah','go'])
#print(mystop_words)
PEOPLE_AWARDS = ['best director - motion picture','best performance by an actress in a television series - comedy or musical','best performance by an actor in a supporting role in a motion picture','best performance by an actress in a supporting role in a series, mini-series or motion picture made for television','best performance by an actress in a motion picture - comedy or musical','best performance by an actress in a television series - drama','best performance by an actress in a motion picture - drama',	'best performance by an actor in a motion picture - comedy or musical','best performance by an actor in a supporting role in a series, mini-series or motion picture made for television','best performance by an actress in a supporting role in a motion picture','best performance by an actor in a mini-series or motion picture made for television','best performance by an actress in a mini-series or motion picture made for television','best performance by an actor in a motion picture - drama', 'best performance by an actor in a television series - drama','best performance by an actor in a television series - comedy or musical']
people_keywords = 'actor|actress|supporting|role|performance|director'
series_keywords = 'series|tv|television'
MOVIE_AWARDS = ['best screenplay - motion picture','best television series - comedy or musical','best original song - motion picture','best motion picture - comedy or musical','best foreign language film','best mini-series or motion picture made for television','best motion picture - drama','best television series - drama','best animated feature film','best original score - motion picture',]

In [5]:
def open_jsonfile(data_path = 'sample_data/gg2013.json'):
  file_ = open(data_path)
  data = json.load(file_)
  return data

In [175]:
def clean_award_name(award_name):
  #removes stopwords from award name and returns key words in award name as list
  award_name = award_name.lower()
  cleaned_award = []
  split = award_name.split()
  for word in split:
    if word not in mystop_words and word not in ['motion','picture','series,','series','role']:  # Many words are irrelevant, removed with list of stopwords
      cleaned_award.append(word)
      if word == 'television':  # Usually television is shortened to tv, want to account for both
        cleaned_award.append('tv')
  return cleaned_award
##############testing function#######################
for award_name in PEOPLE_AWARDS:
  print(clean_award_name(award_name))

['best', 'director']
['best', 'actress', 'television', 'tv', 'comedy', 'musical']
['best', 'actor', 'supporting']
['best', 'actress', 'supporting', 'mini-series', 'television', 'tv']
['best', 'actress', 'comedy', 'musical']
['best', 'actress', 'television', 'tv', 'drama']
['best', 'actress', 'drama']
['best', 'actor', 'comedy', 'musical']
['best', 'actor', 'supporting', 'mini-series', 'television', 'tv']
['best', 'actress', 'supporting']
['best', 'actor', 'mini-series', 'television', 'tv']
['best', 'actress', 'mini-series', 'television', 'tv']
['best', 'actor', 'drama']
['best', 'actor', 'television', 'tv', 'drama']
['best', 'actor', 'television', 'tv', 'comedy', 'musical']


In [188]:
def award_relevant_tweets(tweet_data, award_keywords):
  relevant_tweets = []
  for tweet in tweet_data:
    text = tweet['text']
    text = text.lower()
    tweet_text = tweet['text']
    tweet_text.replace('-',' ')
    tweet_text.replace('/',' ')
    if 'rt @' in text:
      continue
    if 'tv' in award_keywords and not re.search('tv|television|series',text):
      continue
    if 'tv' not in award_keywords and re.search('tv|television|series',text):
      continue
    check = 0
    for key_word in award_keywords:
      if key_word not in text:
        check +=1
    if 'tv' in award_keywords:
      if check <2:
        relevant_tweets.append(tweet_text)
    else:
      if check ==0:
        relevant_tweets.append(tweet_text)
      """else:
        if winner.lower() in text:
          relevant_tweets.append(tweet['text'])"""
  return relevant_tweets

######testing function###########
"""data = open_jsonfile(data_path = 'sample_data/gg2013.json')
award_keywords = clean_award_name('best performance by an actress in a motion picture - drama')
rel_tweets = award_relevant_tweets(data,award_keywords,winner = 'Jessica Chastain')
for rel_tweet in rel_tweets:
  print(rel_tweet)"""


"data = open_jsonfile(data_path = 'sample_data/gg2013.json')\naward_keywords = clean_award_name('best performance by an actress in a motion picture - drama')\nrel_tweets = award_relevant_tweets(data,award_keywords,winner = 'Jessica Chastain')\nfor rel_tweet in rel_tweets:\n  print(rel_tweet)"

In [213]:
def remove_stop_words(small_tweet,mystop_words = mystop_words):
  small_tweet = small_tweet.lower()
  good_tweet = []
  split = small_tweet.split()
  for word in split:
    if re.search('http',word.lower()):
      continue
    if word.lower() not in mystop_words:
      good_tweet.append(word)
  return(' '.join(good_tweet))

###########testing function##########
#small_tweet = 'Golden Globes: Winner for Best Supporting Actor in a Motion Picture is Christoph Waltz for "Django Unchained"'
#print(remove_stop_words('small_tweet here is the link https://asfasdfsd.com'))

In [227]:
def add_to_dictionary(potential_nominees, person_ents,award_keywords,weightage):
  #small_tweet = re.sub('#([a-zA-Z0-9_]{1,50})', '', small_tweet)
  if len(person_ents) == 0:
    return
  for entity in person_ents:
    entity = remove_stop_words(entity)
    check = 0
    """for token in entity.split():
      if token in award_keywords:
        check +=1
    if check == len(entity.split()):
      continue"""
    if len(entity.replace(' ','')) == 0:
      continue
    #entity = entity.replace('#',' ') #removing hashtags symbols
    #entity = re.sub('@([a-zA-Z0-9_]{1,50})', '', entity)
    #entity = re.sub(r'[^\w\s]','',entity) # removing punctuation
    if entity in potential_nominees.keys():
      potential_nominees[entity] += weightage
    else:
      potential_nominees[entity] = weightage

In [60]:
array = ['STEVEN','SPIELBERG','iamgod']
for i in range(len(array)):
  if array[i].isupper():
    array[i] = array[i].capitalize()
print(array)

['Steven', 'Spielberg', 'iamgod']


In [231]:
#text = 'Argo man, what a film. Ben was snubbed for best director at the oscars and Steven did not even win. ridiculous  #goldenglobes'

def return_persons_2_tokens(text,award_keywords):
  array = []
  people = []
  #if 'spielberg' in text.lower():
   # print(text)
  #if len(text.replace(' ','')) == 0:
  #  return people
  text = text.replace('#([a-zA-Z0-9_]{1,50})','') #removing hashtags symbols
  text = re.sub('@([a-zA-Z0-9_]{1,50})', '', text)
  tokens = word_tokenize(text)
  for i in range(len(tokens)):
    if tokens[i].isupper():
      tokens[i] = tokens[i].capitalize()
  pos = pos_tag(tokens)
  ner_tags = ne_chunk(pos, binary=False)
  entity = ""
  for tree in ner_tags.subtrees(filter=lambda t: t.label() == "PERSON"):
    for leaf in tree.leaves():
      if leaf[0].lower() not in award_keywords:
        array.append(leaf[0])
    person = ' '.join(array)
    person = remove_stop_words(person)
    if len(person.split()) == 2:
      people.append(person)
  return people
#print(return_persons_1_tokens(text))

In [11]:
def return_persons_1_tokens(text):
  array = []
  people = []
  if len(text.replace(' ','')) == 0:
    return people
  entity = entity.replace('#',' ') #removing hashtags symbols
  small_tweet = re.sub('@([a-zA-Z0-9_]{1,50})', '', entity)
  tokens = word_tokenize(text)
  pos = pos_tag(tokens)
  ner_tags = ne_chunk(pos, binary=False)
  entity = ""
  for tree in ner_tags.subtrees(filter=lambda t: t.label() == "PERSON"):
    for leaf in tree.leaves():
      array.append(leaf[0])
    if len(array) == 2:
      person = ' '.join(array)
      people.append(person)
    array = []
  return people

In [229]:
def populate_potential_people_nominees(filtered_tweets,award_keywords):
  stretch = 3
  potential_nominees = {}
  prior_keywords = ['won','win','deserve','deserving','deserved','nomination','nominated','snubbed','better','is up for']
  posterior_keywords = ['nominee','deserved award for','preferred','goes to','over','would have been','awarded to','should have','go to','should have been']
  biside_keywords = ['nom','beat','rob','beats','winner','beating','loser','losers','wins over','stole','loses','lost']
  negative_indicating_keywords = 'nominated|nominee|nomination|nominations|nom'
  negative_keywords = 'should have|did not|should\'ve|not|reject|didn\'t'
  for tweet in filtered_tweets:
    text = tweet.lower()
    check = 0
    if re.search(negative_indicating_keywords,text) and re.search(negative_keywords,text):
      continue
    for keyword in prior_keywords:
      if keyword in text:
        check = 1
        small_tweet = tweet[:re.search(keyword,text).start()]
        tokens = small_tweet.split()
        if len(tokens) == 0:
          continue
        elif len(tokens) <= stretch:
          small_tweet = ' '.join(tokens)
          #small_tweet = remove_stop_words(small_tweet)
          person_ents = return_persons_2_tokens(small_tweet,award_keywords)
          add_to_dictionary(potential_nominees,person_ents,award_keywords,weightage = 20)
        else:
          small_tweet = ' '.join(tokens[-stretch:])
          #small_tweet = remove_stop_words(small_tweet)
          person_ents = return_persons_2_tokens(small_tweet,award_keywords)
          add_to_dictionary(potential_nominees,person_ents,award_keywords,weightage = 20)
          small_tweet = ' '.join(tokens[0:-stretch])
          #small_tweet = remove_stop_words(small_tweet)
          person_ents = return_persons_2_tokens(small_tweet,award_keywords)
          add_to_dictionary(potential_nominees,person_ents,award_keywords,weightage = 10)
        #person_ents = return_persons_2_tokens(small_tweet)
        #add_to_dictionary(potential_nominees,person_ents,weightage = 20)
    for keyword in posterior_keywords:
      if keyword in text:
        check = 1
        small_tweet = tweet[re.search(keyword,text).start():]
        tokens = small_tweet.split()
        if len(tokens) <= 1:
          continue
        tokens = tokens[1:]
        if len(tokens) <= stretch:
          small_tweet = ' '.join(tokens)
          person_ents = return_persons_2_tokens(small_tweet,award_keywords)
          add_to_dictionary(potential_nominees,person_ents,award_keywords,weightage = 20)
        else:
          small_tweet = ' '.join(tokens[0:stretch])
          person_ents = return_persons_2_tokens(small_tweet,award_keywords)
          add_to_dictionary(potential_nominees,person_ents,award_keywords,weightage = 20)
          small_tweet = ' '.join(tokens[stretch:])
          person_ents = return_persons_2_tokens(small_tweet,award_keywords)
          add_to_dictionary(potential_nominees,person_ents,award_keywords,weightage = 10)
        #small_tweet = ' '.join(tokens)
        #person_ents = return_persons_2_tokens(small_tweet)
        #add_to_dictionary(potential_nominees,person_ents,weightage = 20)
    for keyword in biside_keywords:
      if keyword in text:
        check = 1
        small_tweet_left = tweet[:re.search(keyword,text).start()]
        small_tweet_right = tweet[re.search(keyword,text).start():]
        tokens_left = small_tweet_left.split()
        tokens_right = small_tweet_right.split()
        if len(tokens_left) > 0:
          if len(tokens_left) <= stretch:
            small_tweet_left = ' '.join(tokens_left)
            person_ents_left = return_persons_2_tokens(small_tweet_left,award_keywords)
            add_to_dictionary(potential_nominees,person_ents_left,award_keywords,weightage = 20)
          else:
            small_tweet_left = ' '.join(tokens_left[-stretch:])
            person_ents_left = return_persons_2_tokens(small_tweet_left,award_keywords)
            add_to_dictionary(potential_nominees,person_ents_left,award_keywords,weightage = 20)
            small_tweet_left = ' '.join(tokens_left[0:-stretch])
            person_ents_left = return_persons_2_tokens(small_tweet_left,award_keywords)
            add_to_dictionary(potential_nominees,person_ents_left,award_keywords,weightage = 10)
        if len(tokens_right) > 1:
          tokens_right = tokens_right[1:]
          if len(tokens_right) <= stretch:
            small_tweet_right = ' '.join(tokens_right)
            person_ents_right = return_persons_2_tokens(small_tweet_right,award_keywords)
            add_to_dictionary(potential_nominees,person_ents_right,award_keywords,weightage = 20)
          else:
            small_tweet_right = ' '.join(tokens_right[0:stretch])
            person_ents_right = return_persons_2_tokens(small_tweet_right,award_keywords)
            add_to_dictionary(potential_nominees,person_ents_right,award_keywords,weightage = 20)
            small_tweet_right = ' '.join(tokens_right[stretch:])
            person_ents_right = return_persons_2_tokens(small_tweet_right,award_keywords)
            add_to_dictionary(potential_nominees,person_ents_right,award_keywords,weightage = 10)
        #person_ents_left = return_persons_2_tokens(small_tweet_left)
        #person_ents_right = return_persons_2_tokens(small_tweet_right)
        #add_to_dictionary(potential_nominees,person_ents_left,weightage = 20)
        #add_to_dictionary(potential_nominees,person_ents_right,weightage = 20)
    if check == 0:
      person_ents = return_persons_2_tokens(tweet,award_keywords)
      add_to_dictionary(potential_nominees,person_ents,award_keywords,weightage = 10)
  return potential_nominees


In [96]:
array = [1,2,3,4,5,6,4,5,6]
print(array[0:-3])

[1, 2, 3, 4, 5, 6]


In [13]:
def get_cosine(vec1, vec2):
    # print vec1, vec2
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    return Counter(WORD.findall(text))

def cosine_similarity(a, b):
    a = text_to_vector(a.strip().lower())
    b = text_to_vector(b.strip().lower())

    return get_cosine(a, b)

##################testing function###############
#print(cosine_similarity('steven ','steven speilberg'))

In [14]:
def char_similarity(a,b):
  final_similarity = 0
  a = a.lower()
  b = b.lower()
  a_tokens = a.split()
  b_tokens = b.split()
  for a_token in a_tokens:
    for b_token in b_tokens:
      similarity = 0
      if len(a_token) > len(b_token):
        bigger_token = a_token
        smaller_token = b_token
      else:
        bigger_token = b_token
        smaller_token = a_token
      for i in range(len(smaller_token)):
        if smaller_token[i] == bigger_token[i]:
          similarity += 1
      if final_similarity < similarity/len(smaller_token):
        final_similarity = similarity/len(smaller_token)
  return final_similarity

##############testing function###################
#print(char_similarity('ang lee','anger'))

In [135]:
def merge_potential_nominees(potential_nominees):
  compressed_nominees = {}
  potential_nominees = sorted(potential_nominees.items(), key = lambda k: -k[1])
  potential_nominees = dict(potential_nominees)
  for key1 in potential_nominees.keys():
    if len(compressed_nominees) == 0:
      compressed_nominees[key1] = potential_nominees[key1]
    else:
      for key2 in list(compressed_nominees.keys()):
        #if cosine_similarity(key1,key2) >= 0.6:
        if len(difflib.get_close_matches(key1,[key2],cutoff = 0.5))==1:
          compressed_nominees[key2] += potential_nominees[key1]
          break
        else:
          if key2 == list(compressed_nominees.keys())[-1]:
            compressed_nominees[key1] = potential_nominees[key1]
  return compressed_nominees

##############testing function###################
#tweet_data = open_jsonfile(data_path = 'sample_data/gg2013.json')
#award_keywords = clean_award_name("best supporting actress in a motion picture")
#relevant_tweets = award_relevant_tweets(tweet_data, award_keywords)
#potential_nominees = populate_potential_movie_nominees(relevant_tweets)
#merged_nominees = merge_potential_nominees(potential_nominees)
#for key in merged_nominees.keys():
#  print(key,merged_nominees[key])
print(difflib.get_close_matches('ang lee',['mind blo'],cutoff = 0.4))

['mind blo']


In [183]:
def check_on_imdb(merged_potential_nominees):
  merged_potential_nominees = sorted(merged_potential_nominees.items(), key = lambda k: -k[1])
  merged_potential_nominees = dict(merged_potential_nominees)
  result_nominees = []
  count = 0
  for key in merged_potential_nominees.keys():
    if len(key.split()) <= 1:
      continue
    persons_identified = ia.search_person(key)
    if len(persons_identified) != 0:
      #print(key,persons_identified[0])
      if (cosine_similarity(key,persons_identified[0]['name'])+char_similarity(key,persons_identified[0]['name']))/2 >= 0.7:
        if len(result_nominees) == 0:
          result_nominees.append(persons_identified[0]['name'])
          count += 1
        elif persons_identified[0]['name'] not in result_nominees:
          result_nominees.append(persons_identified[0]['name'])
          count += 1
      if count == 5 :
        break
  return result_nominees
################testing function####################
#result_nominees = check_on_imdb(merged_nominees)
print(ia.search_person('tommy lee')[0]['name'])

Tommy Lee


In [222]:
def find_nominees(award_name):
  #open the json file of tweets
  tweet_data = open_jsonfile()
  #clean the award name and take it as list of tokens
  award_keywords = clean_award_name(award_name)
  #find all tweets relevant to this award name
  filtered_tweets = award_relevant_tweets(tweet_data,award_keywords)
  #for tweet in filtered_tweets:
  #  print(tweet)
  #print(filtered_tweets)
  #find potential nominees from relevant tweets
  potential_nominees = populate_potential_people_nominees(filtered_tweets,award_keywords)
  #print(potential_nominees)
  #print(potential_nominees)
  #merge potential nominees if there are various version of same nominees
  merged_potential_nominees = merge_potential_nominees(potential_nominees)
  print(merged_potential_nominees)
  #print(merged_potential_nominees)
  #query imdb for best five results
  results = check_on_imdb(merged_potential_nominees)
  #return top five results
  return results
#winners = ['Ben Affleck','Lena Dunham']


In [232]:
find_nominees('best director - motion picture')

{'ben affleck': 2800, 'golden globes': 200, 'oscar snub': 110, 'ang lee': 30, 'jared leto': 20, 'quentin tarantino': 30, 'halle berry': 20, 'steven spielberg': 20, 'mind blo': 20, 'watch ben': 30, 'jennifer garner': 20, 'argo takes': 30, 'motion picture': 30, 'speilberg lincoln': 20, 'joe wright': 10, 'jodie foster': 10, 'matt damon': 10, 'django argo': 20, 'jon hamm': 10}


['Ben Affleck',
 'Golden Glory Team',
 'Oscar Snel',
 'Ang Lee',
 'Quentin Tarantino']

In [233]:
for award_name in PEOPLE_AWARDS:
  print('---------------------------------')
  print(award_name)
  print(find_nominees(award_name))

---------------------------------
best director - motion picture
{'ben affleck': 2800, 'golden globes': 200, 'oscar snub': 110, 'ang lee': 30, 'jared leto': 20, 'quentin tarantino': 30, 'halle berry': 20, 'steven spielberg': 20, 'mind blo': 20, 'watch ben': 30, 'jennifer garner': 20, 'argo takes': 30, 'motion picture': 30, 'speilberg lincoln': 20, 'joe wright': 10, 'jodie foster': 10, 'matt damon': 10, 'django argo': 20, 'jon hamm': 10}
['Ben Affleck', 'Golden Glory Team', 'Oscar Snel', 'Ang Lee', 'Quentin Tarantino']
---------------------------------
best performance by an actress in a television series - comedy or musical
{'lena dunham': 190, 'jennifer lawrence': 20}
['Lena Dunham', 'Jennifer Lawrence']
---------------------------------
best performance by an actor in a supporting role in a motion picture
{'christoph waltz': 1680, 'django unchained': 110, 'golden globes': 100, 'tommy lee': 40, 'alan arkin': 20, 'lee jones': 20, 'leonardo dicaprio': 30, 'garrett hedlund': 20, 'daniel 

In [ ]:
tweet_data = open_jsonfile(data_path = 'sample_data/gg2013.json')
#print(tweet_data[0])
for data in tweet_data:
  if 'rt @' in data['text'].lower():
    continue
  if 'don cheadle' in data['text'].lower() and re.search('best',data['text'].lower())and re.search('actor',data['text'].lower()) and re.search('tv|television',data['text'].lower()) :
    print (data['text'])
    print()